<img src="../../img/weather.png" alt="" style="width: 100%; border-radius: 20px;"/>

## Objective

## Scope

In [1]:
from datetime import datetime, timedelta
from meteostat import Point, Daily

import pandas as pd
import geopandas as gpd
import ee

In [2]:
eea_shapefile_path = r'D:\\Simon\\Documents\\GP\data\\util_files\\eea_grid_files\\eea_europe_grids_50km\\inspire_compatible_grid_50km.shp'
eea_grid = gpd.read_file(eea_shapefile_path)
eea_grid = eea_grid.to_crs('EPSG:4326')

In [3]:
df_path = r'D:\\Simon\\Documents\\GP\\data\\master_validata.csv'
df = pd.read_csv(df_path, index_col=0, low_memory=False)
df = df.head(10)

In [5]:
df['date'] = pd.to_datetime(df['date'])

In [6]:
df.tail(5)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,...,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent
5,6,183,Alpenschneehuhn,2023-05-04,47.50953,10.91606,2002.0,1.0,A1,50kmE4350N2700,...,0.0,0.457885,0.126071,0.0,0.0,0.0,0.0,rocky_area,9,0.0
6,7,183,Alpenschneehuhn,2023-05-06,47.28568,10.18957,1887.0,2.0,B3,50kmE4300N2650,...,0.0,0.000000,0.064984,0.0,0.0,0.0,0.0,grassland,5,0.0
7,8,183,Alpenschneehuhn,2023-05-06,47.27459,10.23008,2109.0,3.0,A1,50kmE4300N2650,...,0.0,0.613074,0.005083,0.0,0.0,0.0,0.0,rocky_area,9,0.0
8,9,183,Alpenschneehuhn,2023-05-06,47.41432,10.34937,1943.0,1.0,A2,50kmE4300N2700,...,0.0,0.000000,0.109643,0.0,0.0,0.0,0.0,grassland,5,0.0
9,10,183,Alpenschneehuhn,2023-05-06,47.41162,10.35219,1919.0,1.0,A2,50kmE4300N2650,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,grassland,5,0.0


In [7]:
df.shape

(10, 29)

## Weather with EE

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-simonbirk')

EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access

In [26]:
def get_weather_data(dataset, date, lat, lon):
    print(date.date())
    point = ee.Geometry.Point([lon, lat])
    image = ee.ImageCollection(dataset).filterDate(date, date).mean()
    data = image.reduceRegion(ee.Reducer.mean(), point, 500)
    print(data)
    data = data.getInfo()
    return data

In [27]:
df['weather_data'] = df.head(10).apply(lambda row: get_weather_data("NOAA/GFS0P25", row['date'], row['coord_lat'], row['coord_lon']), axis=1)

2023-01-20
ee.Dictionary({
  "functionInvocationValue": {
    "functionName": "Image.reduceRegion",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Point",
          "arguments": {
            "coordinates": {
              "constantValue": [
                10.35089,
                47.42037
              ]
            }
          }
        }
      },
      "image": {
        "functionInvocationValue": {
          "functionName": "reduce.mean",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "ImageCollection.load",
                      "arguments": {
                        "id": {
                          "constantValue": "NOAA/GFS0P25"
                      

EEException: Empty date ranges not supported for the current operation.

### Get center of every grid

In [ ]:
germany_switzerland_bbox = eea_grid.cx[5.210942:15.669926, 45.614516:55.379499]
eea_grid_filtered = eea_grid[eea_grid.intersects(germany_switzerland_bbox.unary_union)]
eea_grid_filtered.reset_index(drop=True, inplace=True)

centroid_gdf = eea_grid_filtered.copy()
centroid_gdf['centroid'] = eea_grid_filtered['geometry'].centroid

centroid_gdf['centroid_lon'] = centroid_gdf['centroid'].x
centroid_gdf['centroid_lat'] = centroid_gdf['centroid'].y
centroid_gdf.drop(columns=['geometry', 'noforigin', 'eoforigin', 'gid', 'centroid'], inplace=True)
centroid_gdf.rename(columns={'cellcode': 'eea_grid_id'}, inplace=True)
centroid_gdf

In [ ]:
eea_grid_id = centroid_gdf.eea_grid_id
lat = centroid_gdf.centroid_lat
lon = centroid_gdf.centroid_lon

start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2022-12-31')
# end_date = pd.Timestamp('2018-01-1')
date = pd.date_range(start_date, end_date)


all_combinations = pd.MultiIndex.from_product([date, eea_grid_id], names=['date', 'eea_grid_id'])
all_combinations = pd.DataFrame(index=all_combinations).reset_index()
all_combinations = pd.merge(all_combinations, centroid_gdf[['eea_grid_id', 'centroid_lat', 'centroid_lon']], on='eea_grid_id', how='right')
all_combinations

In [ ]:
def fetch_weather(row):
    location = Point(row.centroid_lat, row.centroid_lon)
    data = Daily(location, row.date, row.date).fetch()
    if data.empty:
        return None
    print(round((row.name / 796136), 3), '%')
    return data.iloc[0]

weather_df = all_combinations.copy()
weather_df[['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']] = weather_df.apply(fetch_weather, axis=1)
weather_df

In [ ]:
from concurrent.futures import ThreadPoolExecutor

num_threads = 8

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    def fetch_weather_concurrent(row):
        location = Point(row['centroid_lat'], row['centroid_lon'])
        data = Daily(location, row['date'], row['date']).fetch()
        if data.empty:
            return None
        return data.iloc[0]

    weather_data_list = list(executor.map(fetch_weather_concurrent, all_combinations.to_dict('records')))

weather_df = all_combinations.copy()
weather_df[['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']] = weather_data_list

In [ ]:
weather_df_path = 'D:\Simon\Documents\GP\Raw_Data\weather_per_day_per_grid.csv'
weather_df.to_csv(weather_df_path)

In [ ]:
# merged_df = df.merge(centroid_gdf[['eea_grid_id', 'centroid_lon', 'centroid_lat']], on='eea_grid_id', how='left')
# merged_df

### Add weather features and merge with dataframe

In [ ]:
# df = pd.merge(df, df_weather, left_index=True, right_index=True)